# Introduction

Zeolites A will be used as the examples in this tutorial. Both crystallize in the cubic crystal system. Zeolite A has a unit cell of $a=24.6100~Å$ (space group: $Fm\bar{3}c$). Zeolites are porous aluminosilicate materials built from corner-sharing SiO$_4$ tetrahedra. The raw data (if not provided) can be downloaded from: [https://zenodo.org/record/1158421](https://zenodo.org/record/1158421). Data were collected using the serial electron diffraction (serialED) data collection routine implemented in the program [instamatic](https://github.com/stefsmeets/instamatic) running on a JEOL JEM-2100-LaB6 at 200 kV equipped with a 512 x 512 Timepix hybrid pixel detector (55 x 55 µm pixel size, QTPX-262k, Amsterdam Scientific Instruments).

This tutorial covers how to explore and process serially collected electron diffraction data (using the serialED technique or otherwise), how to find the crystal orientations and index the data of each individual ED pattern, and how to merge the data to generate a .hkl file that can be used for structure determination (e.g. using [Jana](http://jana.fzu.cz/) or [Shelx](http://shelx.uni-goettingen.de/index.php)). 

A portable installation of the software (appropriately named [problematic](https://github.com/stefsmeets/problematic)) has been provided in the package. The code has been developed in [Python 3](http://python.org), and makes heavy use of the [hyperspy](http://hyperspy.org/) and [pyXem](http://pyxem.github.io/pyxem/) libraries.


# Table of Contents
* [Introduction](#Introduction)
* [Table of Contents](#Table-of-Contents)
* [Getting started with IPython](#Getting-started-with-IPython)
* [Exploring the data](#Exploring-the-data)
* [Importing problematic](#Importing-problematic)
* [Loading data](#Loading-data)
* [Visualization](#Visualization)
* [Data pre-processing](#Data-pre-processing)
  - [Finding the center of the beam](#Finding-the-center-of-the-beam)
  - [Background correction](#Background-correction)
  - [Correction for lens distortions](#Correction-for-lens-distortions)
  - [Image cleaning and peak finding](#Image-cleaning-and-peak-finding)
* [Setting up the indexer](#Setting-up-the-indexer)
* [Index a single pattern](#Index-a-single-pattern)
* [Index all the patterns](#Index-all-the-patterns)
* [Extract and merge intensities](#Extract-and-merge-intensities)
* [Structure determination](#Structure-determination)

# Getting started with IPython

Open up the terminal using `start_Cmder.exe`.

All the processing will be done in a [Jupyter notebook](http://jupyter.org/). Open up a notebook by typing `jupyter notebook` in the terminal. This will open a web interface. Click on `tutorial_zeolite_a.ipynb` to open this tutorial. This tutorial is entirely interactive thanks to [IPython](http://ipython.org/), and runs within the web interface.

In a Jupyter notebook, a command or a group of commands in a cell can be executed by placing the cursor in the cell and pressing 'shift+return'. Try executing the cell below!

In [ ]:
for i in range(3):
    print("Hello Erice!")

Notice the `In [ ]:` changed to `In [1]:`. This means that the cell has been executed! Every cell with this next to it can be executed. This is how we can step through this tutorial interactively. Try changing the text and run the command again by pressing `shift+return`.

# Exploring the data

Before you get started, make sure you have downloaded the data for Zeolite A (if it is not provided, it can be downloaded from https://zenodo.org/record/1158421), and extract it the same directory where is `start_Cmder.exe` (so that it is in `./zeolite_a/`).

The easiest way to get a feel for the data is to simply browse around the images. For this, you can use the browser tool by opening up a new Cmder terminal (a new tab, or running `start_Cmder.exe` again). Then run:

`problematic.browser zeolite_a/images/*.h5`

This will open up a window showing three screens:

<img src="files/browser.png" />

<dt>left</dt><dd>Overview of the grid positions where data have been collected. Every spot indicates a position of the TEM grid where data have been collected. A Large red dot indicates a position where crystals have been detected. Click on a spot to go to open that image.</dd>
<dt>middle</dt><dd>The image corresponding to the stage position shown in the left. This image has been used for locating crystals during the data collection process, and each identified position is indicated with a red cross. The image size represents approximately 6x6 µm. Click on a crystal to open up its diffraction pattern.</dd>
<dt>right</dt><dd>The diffraction pattern corresponding to the crystal selected. The outer edges of the image are at a resolution of approximately $d=0.8~Å$.</dd>

# Importing problematic

To start using *problematic* and process some data, the routines should be imported like any other Python library. Here, we will load the `Indexer` and `Projector` classes that are used for the orientation matching, and the `serialED` class which holds most of the API for loading and processing data. We also import `matplotlib` which is used for plotting. The first line simply determines how the data should be displayed. We will use `tk`, which is suitable for interactive plots.

```
%matplotlib tk
from problematic import Indexer, Projector
from problematic import serialED
import matplotlib.pyplot as plt
```

Don't worry about the warnings thrown, they are just for information.

In [ ]:
%matplotlib tk
from problematic import Indexer, Projector
from problematic import serialED
import matplotlib.pyplot as plt
plt.rcParams["image.cmap"] = "viridis"

# Loading data

There are two data directories in the `zeolite_a` directory, `images` which holds the image data used to locate the crystals, and `data` which contains the diffraction patterns. The diffraction data can be loaded using the `serialED.load` command. This will take a file pattern using wildcards. For example, the pattern `*.img` will match all files with the extension `img` in the current directory. 

Data can be loaded using:

`serialED.load("zeolite_a/data/image_*.h5"`

where `*` is the wildcard that matches all files that match this pattern. Since the entire data set is quite large (approximately 1-1.5~GB of data will be loaded into memory), data processing later on will take a while. Therefore, for the purpose of this tutorial, we will use a reduced data set.

You may already have noticed that not all diffraction patterns are of equal quality. The file `filelist.txt` contains a list of files identified by a deep convoluted neural network as *good quality* diffraction data.

The filelist can be read in like this:

In [ ]:
with open("filelist_a.txt") as f:
    filelist = [line.strip() for line in f]

Because the data set is still quite large, we will work on the first 100 images

In [ ]:
filelist = filelist[0:100]

To load the data:

In [ ]:
# ed = serialED.load("zeolite_a/data/image_0[0-1]*.h5")
ed = serialED.load(filelist)
ed.data.shape

**Note**: if you are unsure what a command does, in Jupyter you can pre/apppend a question mark `?` to get some more information. Try running `serialED.load?`.

In [ ]:
serialED.load?

Let's have a look at what is inside the data variable.

In [ ]:
ed

The `serialED.load` command has found all the diffraction patterns in the `zeolite_a` directory, opened them, and stored them in the loaded them, and placed them into the `ed` variable. This is were we make heavy use of the *hyperspy* library, which does all the heavy lifting underneath.

The `|` symbol separates the navigation dimensions and the signal dimensions. This is essentially a *stack* of images ranging from 0 to 1107 (the *navigation* dimension), where each image has a resolution of 516 by 516 pixels (the *signal* dimensions). This allows us to efficiently iterate over these data.

# Visualization

To visualize the data, use the `plot` method. This will pop up two windows (sometimes in the background). If the contrast is not right, try changing the value of `vmax`. One window (*serialED Navigator*) simply shows the sum of the data in the image. You can move the cursor around in this image to select the different frames or use the arrow keys. The other window (*serialED Signal*) shows the currently selected diffraction pattern.

**Note**: the window may have pop up in the background!

In [ ]:
ed.plot(vmax=300)

# Data pre-processing

Before the data can be indexed, there are several steps that should be executed to prepare the data.

 * Finding the center of the beam
 * Background subtraction
 * Correction for lens distortions
 * Image cleaning and peak finding

## Finding the center of the beam

Despite exhaustive efforts to calibrate the position of the direct beam using the condensor and projector lens alignment coils, the position of the primary beam usually moves around within 10-20 pixels. Therefore, the position of the direct beam should be determined for every pattern to apply some of the corrections and for the indexing routine.

The position of the direct beam is found by applying a Gaussian filter to the entire diffraction pattern with a large
enough standard deviation (`sigma`, usually 10-30). The position of the direct beam will then be at the pixel with the largest intensity.

In [ ]:
sigma = 10
centers = ed.get_direct_beam_position(sigma=sigma)

## Background correction

Background subtraction for peak identification is performed using a median filter. The window for the filter was chosen to be 19 pixels wide (as defined by `footprint`), which defines a box of approximately three times the number of pixels in a peak. This box is then scanned over the image, and the median pixel intensity is taken for the background. The window for the filter should  which defines a box of approximately three times the number of pixels in a peak.

In [ ]:
footprint = 19
processed = ed.remove_background(footprint=footprint)

This will create a new object with the background-subtracted data stored in the `processed` variable to avoid overwriting the original data. This is in case we want to change the `footprint`. Feel free to experiment and try different values. The effect of the background subtraction can be visualized using the `.plot` method.

In [ ]:
processed.plot(vmax=300)

## Correction for lens distortions

Lens distortions are corrected by applying an affine transformation to the image. On our microscope, we observe an elliptical distortion with an eccentricity of 0.22. These are described by the `azimuth` (the direction of the major axis of the ellipse) and the `amplitude` (the percentage difference in length between the major and minor axis).

In [ ]:
azimuth = -6.61
amplitude = 2.43
processed = processed.apply_stretch_correction(azimuth=azimuth, 
                                               amplitude=amplitude, 
                                               centers=centers)

## Image cleaning and peak finding

Peak detection is then performed by calculating the difference between two gaussian convolutions of the input image with standard deviations of `sigma_min` and `sigma_max`. Their values are usually somewhere between 1 and 10, making sure that `sigma_max` is larger than `sigma_min`. Reflections are then identified by searching for regions of connected pixels that have an intensity value larger than a `threshold` (usually 1 to 3). The intensities of the the pixels belonging to the background or any regions consisting of less than `min_size` (usually 20 or more).

Because this operation is quite critical to the success of the indexing algorithm, `processed.find_peaks_interactive` command can be used to interactively tweak the numbers. Select the method *find_peaks_regionprops*. The image can be changed using the coordinate input, or the randomize button. Drag the slider to change the values until you are satisfied. 

**Note**: The spinboxes on the sides of the sliders simply adjust the limits for the slider. The current value is shown to the right of the slider.

In [ ]:
processed?

In [ ]:
# change the value of vmax if the contrast is not right
processed.find_peaks_interactive(imshow_kwargs={"vmax": 100})

Once you are satisfied, save the numbers into the variables below. Otherwise, the values given below are a good starting point.

In [ ]:
min_sigma=4
max_sigma=5
threshold=1
min_size=50
processed = processed.find_peaks_and_clean_images(min_sigma=min_sigma, 
                                                  max_sigma=max_sigma, 
                                                  threshold=threshold, 
                                                  min_size=min_size)

This would be a good time to save the current progress.

In [ ]:
processed.save("processed.hdf5")

The data (including the beam centers) can be loaded back in using the following command. In case you need to restart the notebook, you can continue from the next cell (don't forget the imports!).

In [ ]:
processed = serialED.load("processed.hdf5")
centers = processed._centers

# Setting up the indexer

To index these data requires *a priori* knowledge of the unit cell and space group. First, a `Projector` object should be created. `dmin` and `dmax` indicate the resolution range. `thickness` affects the shape function of the reflections, also known as the excitation error at which the reflections are still visible. A thicker crystal has sharper (better defined) reflections, and therefore a smaller excitation error (= less reflections).

In [ ]:
name = "LTA"
spgr = "Fm-3c"
params = (24.6100,)

dmin, dmax = 1.0, 10.0
thickness = 400

projector = Projector.from_parameters(params, 
                                      spgr=spgr, 
                                      name=name, 
                                      dmin=dmin, 
                                      dmax=dmax, 
                                      thickness=thickness)

This will generate a `Projector` object that is used to generate a library of all possible orientations. The projection is defined by polar coordinates $\alpha$ and $\beta$ that determine the zone-axis, and combined with in-plane rotation $\gamma$ (around the zone-axis) to give the crystal orientation. Note that the values given are radians. `show_hkl` can be set to `False` to toggle the display of the *hkl* labels.

In [ ]:
plt.close("all")  # this line closes the open plot (if any)

projector.plot(alpha=0.3, beta=0.2, gamma=0.1, show_hkl=True)

In a similar way, we can also define the zone-axis directly.

In [ ]:
plt.close("all")

zone_axis = (0, 1, 0)
projector.plot_along_axis(zone_axis=zone_axis, gamma=0.1, show_hkl=True)

This makes it possible to easily generate all possible diffraction patterns, and compare them *one-by-one* with the observed data. This can be done using the method `Projector.generate_all_projections` is used. This function reads a list of unique crystallographic orientations ($\alpha$ and $\beta$) that have been generated using the matlab library [MTEX](http://mtex-toolbox.github.io/). All orientations are spaced 0.03 radians apart (about 1.7°) to ensure a balance between a reasonable number of orientations and a dense enough coverage. When combined with the in-plane rotation $\gamma$, this means that for Laue group $\bar{1}$ about 1.5 million crystal orientations are generated.

These are mostly the details of the implementation, and the `Indexer` class takes care of most of this. Initialize the `Indexer` by passing the `projector` object and the `pixelsize` (given in $\mathrm{px}/Å$). Although the pixelsize can be refined later, it is important to have a good starting value. It should also be noted that because a convergent beam is used for the experiment, the calibrated pixelsize on the microscope can be a bit off.

In [ ]:
pixelsize = 0.003957  # pixels per angstrom
indexer = Indexer.from_projector(projector, pixelsize=pixelsize)

Because of the high symmetry from the cubic system, the number of orientations generated is significantly reduced, which will speed up the data processing. 

# Index a single pattern

Now that we have:

1. Loaded the data
2. Explored the data
3. Found the center of the diffraction patterns
4. Processed the data to isolate the reflection signal
5. Generated a library of all possible crystallographic orientations
6. Set up the indexer

it is finally time to start the indexing procedure. This can be done using the `find_orientation` function, which takes an image and the *beam center* as arguments, and optionally the number of orientations to be returned (`nsolutions`).

In [ ]:
i = 19
im = processed.data[i]
# im = ed.data[i]  # to see what happens with the raw data
c = centers.data[i]
orientations = indexer.find_orientation(im, c, nsolutions=25)

This will return a list of the 25 orientations that best match the diffraction pattern. This is determined by the *score* of the orientation, which is essentially defined as the sum of the intensities at each of the pixel positions generated by the `Projector`. It also accounts for how far the reflection is from the Bragg condition, and the ratio of present/absent reflections in the pattern.

The 25 orientations are defined by  an `IndexingResult` object that contains all the information to uniquely identify an orientation. This includes the *score*, and all the information required to reproduce the orientation and corresponding projection.

In [ ]:
# Change the number in square brackets to look at the different orientations
# Note that in Python, counting starts at 0!

orientations[0]

More useful is to have a look at what this orientation actually looks like by visualizing it. You can change the number in the brackets to have a look at the different solutions. The idea is that the best solution (highest *score*) is the correct one! You can use the processed or the raw data for the visualization. Simply replace `raw` with `im`.

In [ ]:
plt.close("all")

raw = ed.data[i]
indexer.plot(raw, orientations[0])

Notice that the match is quite good, but that it is not perfect. The reason for this is two-fold. First, remember how the orientations are spaced roughly 0.03 radians apart. The ideal solution may be inbetween two orientations, or the starting *pixelsize* or *beam center* may be somewhat off. Second, the model to generate the diffraction patterns is fairly basic and does not account for all the factors that affect the position of the reflections. However, this is not such a big problem, as long as it is possible to match a reflection to its indices.

To improve the orientation, we can perform a least-squares minimization. We can choose to vary the different parameters if necessary, although sometimes refining everything at once is not stable, and the orientations have to be refined one step at the time.

In [ ]:
plt.close("all")

refined = indexer.refine(im, orientations[0], 
                         method="powell", 
                         vary_center=True, 
                         vary_scale=True, 
                         vary_alphabeta=True, 
                         vary_gamma=True)

indexer.plot(im, refined)

refined

Hopefully, you will notice that the fit is slightly better. Notice how the score has significantly increased, indicating a better fit to the data. Feel free, to play around by selecting different images and different settings for the refinementn to get a feel for the data, or continue to the next cell!

# Index all the patterns

Although it is possible to process the the entire data set by hand, this is a little bit time consuming. To help with this, the `serialED.find_orientations` method can be used. What this does is run `Indexer.find_orientation` on all diffraction pattern in the data set. `find_orientations` takes an `Indexer` object and a list of *beam centers*.

In [ ]:
orientations = processed.find_orientations(indexer, centers, nsolutions=25)

A list of orientations can be saved:

In [ ]:
serialED.io_utils.save_orientations(orientations, out="orientations.npy")

and loaded back in:

In [ ]:
orientations = serialED.io_utils.load_orientations("orientations.npy")

To explore the best orientations, you can use the `orientation_explorer` to see the highest ranked orientation solution for each diffraction pattern.

In [ ]:
ed.orientation_explorer(indexer, orientations, imshow_kwargs={"vmax":200, "cmap":"gray"})

The orientations have not been refined yet at this stage. A similar method exists for refining all orientations. To reduce the processing time a bit, the fit tolerance is increased to 1.0 and the center is not refined. This should increase the speed by a factor 5. If it still takes too long, you can interrupt the current operation pressing the ⬛-button (black square) in the menu bar, and load the previously generated orientations.

In [ ]:
refined = processed.refine_orientations(indexer, orientations, vary_center=False, fit_tol=1.0)
serialED.io_utils.save_orientations(refined, out="refined.npy")

In case the last step failed or was interrupted, you can load the previously generated orientations by uncommenting (removing the "#") and running the following cell:

In [ ]:
# refined = serialED.io_utils.load_orientations("tutorial_first100.npy")

Again, you can use the `orientation_explorer` to see the highest ranked orientation solution for each diffraction pattern.

In [ ]:
ed.orientation_explorer(indexer, refined, imshow_kwargs={"vmax":200, "cmap":"gray"})

A list of all current orientations can be exported. This will generate a combined yaml/csv file with all the orientation data and phase information.

In [ ]:
# note that the orientations keyword is optional, and can be omitted if the refinement step was succesful
processed.export_indexing_results(fname="orientations.ycsv", orientations=refined)

# Extract and merge intensities

By now, you may have noticed that not all crystals diffraction equally well and that not all diffraction patterns can be indexed well, for example, because of multiple crystals overlapping, crystals being too thick, or crystals simply being of poor quality. This is expected. In a regular TEM experiment, we always look for the very best crystals, and the ones that are not of good enough quality are discarded. This idea also applies to the indexing and merging strategy. 

The indexing process is therefore used as a filter. Because we expect that not all crystals diffract equally well, we can also expect that not all diffraction patterns are indexed equally well. Luckily, the *indexing score* gives us an idea of the quality of a crystal and its diffraction pattern.

First, let's extract the intensities. This is done using the following command, which takes any orientation list and an `Indexer` object as arguments. The intensities correspond to the largest intensity within a 3 pixel radius around the predicted spot

In [ ]:
intensities = processed.extract_intensities(orientations=refined, indexer=indexer)

Even if we select the best diffraction patterns, errors in the data may still occur, such as misindexed diffraction patterns (for reasons described above). The merging strategy takes care of this (dubbed SerialMerge, for details see [J. Appl. Cryst. (2017). 50, 885-892](https://doi.org/10.1107/S1600576717005854)).

Instead of modelling the diffraction processes that relate the structure factors to the intensities of the spots observed in the diffraction pattern, the SerialMerge method retrieves the most likely reflection ranking. As a result, it is tolerant to errors in the diffraction intensities, which are inevitable in electron diffraction because of dynamical scattering, and the issue of scaling is avoided altogether. The downside is that the algorithm produces the most likely ranking of the reflections only; the values of the reflection intensities are lost in the merging procedure (because it optimizes for rank, not intensity).

In the cell below, the 50 best diffraction patterns are selected and saved to the file *merged.hkl*.

In [ ]:
m = serialED.serialmerge_intensities(intensities, refined, n=100)

As mentioned above, the intensity information is lost in the merging process. It is well known that for structure determination, the actual value of the intensities is not critical, as long as strong reflections are strong and weak reflections are weak. However, to recover a sensible distribution of intensities, a histogram matching routine can be applied, where the observed or calculated intensities from a related material may be used as a source of intensities or structure factors. This can improve the quality of the merged data.

This can be done using the function below. Here, we are matching the histogram of the merged data to that of archetypical aluminosilicate zeolite ZSM-5 (**MFI**). The file will be saved to the file `matched.hkl`.

In [ ]:
serialED.match_histogram(data="merged.hkl", histogram="files/MFI.hkl", fout="matched.hkl")

# Structure determination

The data processing above has been limited to the first 100 images. Unfortunately, the data are not complete enough for structure determination. You can run the code below to load the entire data set of previously processed data, including the refined orientations.

In [ ]:
processed_all = serialED.load("files/processed_all.hdf5")
orientations_all = serialED.io_utils.load_orientations("files/refined_all.npy")

And extract and merge the intensities:

In [ ]:
intensities = processed_all.extract_intensities(orientations=orientations_all, indexer=indexer)
m = serialED.serialmerge_intensities(intensities, orientations_all, n=200, fout="merged_all.hkl")
serialED.match_histogram(data="merged_all.hkl", histogram="files/MFI.hkl", fout="lta.hkl")

In [ ]:
processed_all.plot(vmax=100)

Now that you have an hkl file, you can attempt to solve the crystal structure! Feel free to use the software of your choice. 

Run the cell below to generate an input file for shelxs.

In [ ]:
with open("lta.ins", "w") as f:
    print("""TITL lta in Fm-3c #226
CELL 0.0251 24.61 24.61 24.61 90 90 90
ZERR 192 0 0 0 0 0 0
LATT 4
SYMM -Y,+X,0.5+Z
SYMM -X,-Y,+Z
SYMM +Y,-X,0.5+Z
SYMM +X,-Z,0.5+Y
SYMM +X,-Y,-Z
SYMM +X,+Z,0.5-Y
SYMM +Z,+Y,0.5-X
SYMM -X,+Y,-Z
SYMM -Z,+Y,0.5+X
SYMM +Z,+X,+Y
SYMM +Y,+Z,+X
SYMM -Y,-Z,+X
SYMM +Z,-X,-Y
SYMM -Y,+Z,-X
SYMM -Z,-X,+Y
SYMM -Z,+X,-Y
SYMM +Y,-Z,-X
SYMM +Y,+X,0.5-Z
SYMM -Y,-X,0.5-Z
SYMM -X,+Z,0.5+Y
SYMM -X,-Z,0.5-Y
SYMM +Z,-Y,0.5+X
SYMM -Z,-Y,0.5-X
SFAC Si O
UNIT 192 384

TREF
HKLF 4

END
""", file=f)

If you have managed to find a structure that looks more or less like this:

<img src="files/lta.png" align="bottom" style="max-height: 300; max-width: 300px;" />

Well done, you are awesome! You have solved the crystal structure of zeolite A and reached the end of this tutorial!

(If you are eager for more, you can try to run through the data set of zeolite Y. You can open a new notebook by clicking`File` > `new` > `Py3 (serialED)`, which has already been set up for data processing.)